In [ ]:
from torch.utils.data import TensorDataset, DataLoader

mode = "freeze"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = 1 #train_labels.shape[1]
clip_classifier = CLIPClassifier(model, mode, num_labels)
clip_classifier = clip_classifier.to(device)
if mode == "freeze":
    train_ds = TensorDataset(train_clip_feats, 
                             torch.from_numpy(train_labels)[:, 0].unsqueeze(1))
    val_ds = TensorDataset(val_clip_feats, 
                           torch.from_numpy(val_labels)[:, 0].unsqueeze(1))
    test_ds = TensorDataset(test_clip_feats, 
                                  torch.from_numpy(test_labels)[:, 0].unsqueeze(1))
elif mode == "train_norm":
    # create image datasets
    pass

In [ ]:
train_losses = []
val_losses = []
val_accs = []
val_aps = []

val_dl = DataLoader(val_ds, num_workers=0, shuffle=False, batch_size=2048)
test_dl = DataLoader(test_ds, num_workers=0, shuffle=False, batch_size=2048)

val_step(clip_classifier, val_dl)


In [ ]:
n_epochs = 5
bs = 2048

train_dl = DataLoader(train_ds, num_workers=0, shuffle=True, batch_size=bs)



optimizer = torch.optim.AdamW(clip_classifier.parameters(),
                             lr=5e-5,#5e-5,
                             betas=(0.9, 0.98),
                             eps=1e-6,
                             weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

loss_func = torch.nn.BCEWithLogitsLoss()

import time
start_time = time.time()

max_steps = 10000
log_every = 30
step = 0
pbar = tqdm(total=max_steps)
stop = False

while not stop:
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        # calc classifier loss
        preds = clip_classifier(x)
        loss = loss_func(preds, y)
        
        # backward
        optimizer.zero_grad()
        #for p in clip_classifier.parameters():
        #    p.grad = None
        loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            #convert_models_to_fp32(model)
            optimizer.step()
            #clip.model.convert_weights(model)
        # log
        episode_train_losses.append(loss.detach())
        step += 1
        pbar.update(1)
        
        if step % log_every == 0:
            train_losses.append(torch.stack(episode_train_losses).mean().item())
            val_step(clip_classifier, val_dl)

            episode_train_losses = []

        if step >= max_steps:
            stop = True
            break
print("Total seconds: ", time.time() - start_time)

In [ ]:
plt.plot(range(1, len(train_losses) + 1), train_losses, label="train_loss")
plt.plot(range(0, len(train_losses) + 1), val_losses, label="val_loss")
plt.legend()

In [ ]:
plt.plot(val_aps)
print(max(val_aps))

In [ ]:
plt.plot(val_accs)
print(max(val_accs))